# Data Cleaning and Preparation with Pandas

This notebook is dedicated solely to loading, cleaning, and preparing the job dataset. 
It performs the following tasks:

• Loading and appending multiple CSV files.

• Resetting the dataframe index.

• Parsing the 'Years of experience' column into numeric minimum and maximum values.

• Cleaning the 'Salaries' column (removing currency markers, handling non-numeric rows, and parsing numeric values).

• Removing extraneous characters from the 'Skills' and 'Locations' columns.

• Reordering the columns.

After cleaning, a utility function get_df() is provided to return the fully cleaned dataset.



In [1]:
import os
import pandas as pd


## Load and Append CSV Files

This cell reads multiple CSV files and appends them into a single dataframe.



In [2]:
# Initialize an empty DataFrame.
df = pd.DataFrame()

# Append CSV files.
for i in range(1, 8):
    csv_path = rf"D:\mozakra\Python\PycharmProjects\JobDataExtractionAndAnalysis\Data\page{i}.csv"
    if os.path.exists(csv_path):
        csv_reader = pd.read_csv(csv_path)
        # Append each CSV and ignore the original index.
        df = df._append(csv_reader, ignore_index=True)
    else:
        print(f"Warning: File not found at {csv_path}")



## Reset the DataFrame Index

Reset the index to ensure a sequential index.



In [3]:
df.reset_index(drop=True, inplace=True)


## Clean and Parse the Experience Column

We adjust and split the 'Years of experience' column into two separate numeric columns:

• 'Min Years of Experience'

• 'Max Years of Experience'



In [4]:
# Insert a space after 'Years' to guarantee proper splitting.
df['Years of experience'] = df['Years of experience'].str.replace('Years', ' Years')

# Create numeric experience columns.
df['Min Years of Experience'] = df['Years of experience'].str.split().str[0].astype("int64")
df['Max Years of Experience'] = df['Years of experience'].str.split().str[2].astype("int64")



## Clean the Salaries Column

Cleaning steps:
1. Ensure only rows with numeric salary data are preserved.
2. Remove the 'USD ' currency marker.
3. Split the salary string and convert the two parts to numeric values.

Rows without valid numeric data are set to missing.



In [5]:
# Create a mask to check for numeric content.
mask = df['Salaries'].str.contains(r'\d', regex=True, na=False)
df.loc[:, 'Salaries'] = df['Salaries'].where(mask, pd.NA)

# Reset index in case of dropped rows.
df.reset_index(drop=True, inplace=True)

# Remove the currency marker.
salary_str = df['Salaries'].str.replace('USD ', '', regex=False)

# Convert the split strings to floats safely.
df['Min Salary'] = pd.to_numeric(salary_str.str.split().str[0], errors='coerce')
df['Max Salary'] = pd.to_numeric(salary_str.str.split().str[2], errors='coerce')



## Clean Skills and Locations Columns

Remove unwanted characters (e.g., brackets and quotes) from the Skills and Locations fields.



In [6]:
# Clean up the Skills column.
df['Skills'] = (df['Skills']
                .str.replace("[", "", regex=False)
                .str.replace("]", "", regex=False)
                .str.replace("'", "", regex=False))

# Clean up the Locations column.
df['Locations'] = (df['Locations']
                   .str.replace("[", "", regex=False)
                   .str.replace("]", "", regex=False)
                   .str.replace("'", "", regex=False))



## Reorder Columns

Reorder the dataframe columns into an established order for consistency.



In [7]:
NewOrder = ["Company Names",
            "Locations",
            "Years of experience",
            "Min Years of Experience",
            "Max Years of Experience",
            "Skills",
            "Salaries",
            "Min Salary",
            "Max Salary",
            "Links"]
df = df.reindex(columns=NewOrder)



## Utility Function to Retrieve Clean Data


In [8]:
def get_df():
    """
    Returns a fully cleaned dataframe with complete rows.
    
    Returns:
        pd.DataFrame: Clean and prepared job data.
    """
    return df.dropna()


# Display DataFrame after Cleaning

In [9]:
df

,Company Names,Locations,Years of experience,Min Years of Experience,Max Years of Experience,Skills,Salaries,Min Salary,Max Salary,Links
0,Sparks To Ideas,Ahmedabad (Gujarat),0 - 1 Years,0,1,"Core Python, Numpy, Pycharm, DJango, Machine L...",2.05 - 6.10 Lakhs,2.05,6.1,https://www.timesjobs.com/job-detail/hiring-fo...
1,LTIMindtree,Pune (Maharashtra),8 - 12 Years,8,12,Mandatory Skills : Dimensional Data ModelingGo...,<NA>,NaN,NaN,https://www.timesjobs.com/job-detail/data-scie...
2,World wide Immigration Service Llp,Luxembourg (Luxembourg),3 - 8 Years,3,8,"network engineer, network administrator, Netwo...",50.00 - 95.00 Lakhs,50.00,95.0,https://www.timesjobs.com/job-detail/network-e...
3,LTIMindtree,Pune (Maharashtra),8 - 12 Years,8,12,Mandatory Skills : Dimensional Data ModelingGo...,<NA>,NaN,NaN,https://www.timesjobs.com/job-detail/data-scie...
4,TALPRO INDIA PRIVATE LIMITED,Bengaluru/ Bangalore (Karnataka),5 - 15 Years,5,15,Python,15.30 - 30.60 Lakhs,15.30,30.6,https://www.timesjobs.com/job-detail/python-de...
...,...,...,...,...,...,...,...,...,...,...
163,LAKSH HUMAN RESOURCE,Mumbai (Maharashtra),1 - 3 Years,1,3,"rest, python, django, git",<NA>,NaN,NaN,https://www.timesjobs.com/job-detail/python-de...
164,zenga tv,Gurgaon (Haryana),2 - 5 Years,2,5,"python, django, html5, javascript",<NA>,NaN,NaN,https://www.timesjobs.com/job-detail/python-de...
165,SEVEN CONSULTANCY,"Navi Mumbai (Maharashtra), Mumbai (Maharashtra...",1 - 3 Years,1,3,"rest, python, storage",<NA>,NaN,NaN,https://www.timesjobs.com/job-detail/python-de...
166,INFINITY GROUP,Noida/ Greater Noida (Uttar Pradesh),2 - 5 Years,2,5,"python, css, django, html, bootstrap",<NA>,NaN,NaN,https://www.timesjobs.com/job-detail/python-de...



# End of Data Cleaning Notebook

The clean dataset can now be used in further analysis and visualizations.

